# Module 3: The CodeAct Pattern

**Duration:** ~15 minutes  
**Goal:** Understand how agents generate and execute custom Python code

---

## The Limitation of Pre-Built Tools

In Module 1, we gave the agent tools. In Module 2, we saw it reason through multi-step problems.

But here's a question those tools can't answer:

```
"Calculate the Sharpe ratio for a portfolio with these monthly returns:
 5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%"
```

We don't have a `calculate_sharpe_ratio` tool. We could build one, but what about tomorrow's question?

What if someone asks for the **Sortino ratio**? Or **Value at Risk**? Or a **custom metric** you've never heard of?

**You can't pre-build tools for every possible calculation.**

This is where **CodeAct** comes in.

## What is CodeAct?

CodeAct is a pattern where the agent doesn't just call pre-built tools — it **writes and executes Python code on the fly**.

```
┌──────────────────────────────────────────────────────────┐
│                    CODEACT PATTERN                        │
│                                                           │
│   USER                                                    │
│   "Calculate Sharpe ratio"                                │
│            │                                              │
│            ▼                                              │
│   ┌─────────────────┐                                    │
│   │   LLM THINKS    │                                    │
│   │ "I need to      │                                    │
│   │  write code     │                                    │
│   │  for this"      │                                    │
│   └────────┬────────┘                                    │
│            │                                              │
│            ▼                                              │
│   ┌─────────────────┐     ┌─────────────────┐           │
│   │  LLM WRITES     │ ──→ │   PYTHON        │           │
│   │  PYTHON CODE    │     │   EXECUTES      │           │
│   │                 │     │   THE CODE      │           │
│   │  import numpy   │     │                 │           │
│   │  returns = ...  │     │   Result: 0.72  │           │
│   │  sharpe = ...   │     │                 │           │
│   └─────────────────┘     └─────────────────┘           │
│                                                           │
└──────────────────────────────────────────────────────────┘
```

The agent:
1. Reasons about what code to write
2. Writes the Python code
3. The system executes it
4. The agent sees the result and can continue reasoning

**The agent isn't limited to pre-built tools. It can compute anything Python can compute.**

---

## Setup

In [1]:
# Install smolagents if needed
!pip install smolagents -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel
import getpass

In [3]:
# Enter your API key
API_KEY = getpass.getpass("Enter your OpenAI API key: ")
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model ready!")

Enter your OpenAI API key: ··········
Model ready!


---

## CodeAct in Action

**Notice something important:** We're not passing any tools. The agent's only capability is writing and executing Python.

In [4]:
# CodeAgent is specifically designed for code generation and execution
agent = CodeAgent(
    tools=[],  # No pre-built tools!
    model=model,
    verbosity_level=LogLevel.INFO,
    additional_authorized_imports=["numpy", "pandas"]  # Allow these imports
)

print("CodeAgent ready with NO tools — only code generation!")

CodeAgent ready with NO tools — only code generation!


---

## The Sharpe Ratio Example

Let's ask that Sharpe ratio question. Watch how the agent:
1. Thinks about what formula to use
2. Writes the Python code
3. Executes it and interprets the results

In [5]:
result = agent.run("""
Calculate the Sharpe ratio for a portfolio with these monthly returns:
5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%

Assume a risk-free rate of 0.5% per month.
Show your calculation steps.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the Sharpe ratio for a portfolio with these monthly returns:                                          │
│ 5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%                                                                                │
│                                                                                                                 │
│ Assume a risk-free rate of 0.5% per month.                                                                      │
│ Show your calculation steps.                                                                                    │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
                                                                                                                   
  # Monthly returns and risk-free rate                                                                             
  returns = np.array([5, 3, -2, 4, 1, -1, 3, 2]) / 100  # Convert percentage to decimal                            
  risk_free_rate = 0.5 / 100  # Convert risk-free rate to decimal                                                  
                                                                                                                   
  # Step 1: Calculate the average return                                                                           
  average_return = np.mean(returns)                                                                                
  print("Average Return:", average_return)                                                                         
                                                                                                                   
  # Step 2: Calculate the standard deviation of the returns                                                        
  std_deviation = np.std(returns)                                                                                  
  print("Standard Deviation:", std_deviation)                                                                      
                                                                                                                   
  # Step 3: Calculate the excess return                                                                            
  excess_return = average_return - risk_free_rate                                                                  
  print("Excess Return:", excess_return)                                                                           
                                                                                                                   
  # Step 4: Calculate the Sharpe Ratio                                                                             
  sharpe_ratio = excess_return / std_deviation                                                                     
  print("Sharpe Ratio:", sharpe_ratio)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Average Return: 0.018750000000000003
Standard Deviation: 0.022603926650031407
Excess Return: 0.013750000000000002
Sharpe Ratio: 0.6083013899702642

Out: None

[Step 1: Duration 7.01 seconds| Input tokens: 2,032 | Output tokens: 310]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(0.6083013899702642)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 0.6083013899702642

[Step 2: Duration 2.50 seconds| Input tokens: 4,686 | Output tokens: 407]

**What just happened:**
- The agent **wrote the code** to calculate Sharpe ratio
- It **executed** it and got results
- It **explained** the results in plain English

We didn't build a Sharpe ratio tool. **The agent knew the formula and implemented it on the spot.**

---

## Why This is Powerful

Think about what just happened:

- **The LLM brought knowledge** — it knew the Sharpe ratio formula
- **Python brought computation** — it did the actual math
- **CodeAct combined them** — knowledge + execution

This is why CodeAgent is so useful for finance. Financial analysis often involves **custom calculations**. You can't pre-build everything. But the agent can figure it out.

---

## More Examples

Let's try a few more to see the range of what's possible.

In [6]:
# Example 2: Maximum Drawdown
result = agent.run("""
Calculate the maximum drawdown for this sequence of portfolio values:
$100,000 → $105,000 → $98,000 → $102,000 → $95,000 → $110,000

Explain what maximum drawdown means and show the calculation.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the maximum drawdown for this sequence of portfolio values:                                           │
│ $100,000 → $105,000 → $98,000 → $102,000 → $95,000 → $110,000                                                   │
│                                                                                                                 │
│ Explain what maximum drawdown means and show the calculation.                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  portfolio_values = [100000, 105000, 98000, 102000, 95000, 110000]                                                
                                                                                                                   
  max_drawdown = 0                                                                                                 
  peak = portfolio_values[0]                                                                                       
                                                                                                                   
  for value in portfolio_values:                                                                                   
      if value > peak:                                                                                             
          peak = value                                                                                             
      drawdown = (peak - value) / peak                                                                             
      if drawdown > max_drawdown:                                                                                  
          max_drawdown = drawdown                                                                                  
                                                                                                                   
  max_drawdown_percentage = max_drawdown * 100  # Convert to percentage                                            
  print(f"The maximum drawdown is {max_drawdown_percentage:.2f}%")                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The maximum drawdown is 9.52%

Out: None

[Step 1: Duration 4.80 seconds| Input tokens: 2,029 | Output tokens: 266]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(9.52)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 9.52

[Step 2: Duration 1.81 seconds| Input tokens: 4,518 | Output tokens: 352]

**Maximum drawdown** — a metric you'd definitely want in any portfolio analysis. The agent knew what it was, wrote the code, and calculated it.

In [7]:
# Example 3: Correlation analysis
result = agent.run("""
Given these monthly returns for two stocks:

Stock A: 2%, -1%, 3%, 1%, -2%, 4%
Stock B: 1%, 0%, 2%, 2%, -1%, 3%

Calculate the correlation coefficient between them.
Explain whether they would provide good diversification.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Given these monthly returns for two stocks:                                                                     │
│                                                                                                                 │
│ Stock A: 2%, -1%, 3%, 1%, -2%, 4%                                                                               │
│ Stock B: 1%, 0%, 2%, 2%, -1%, 3%                                                                                │
│                                                                                                                 │
│ Calculate the correlation coefficient between them.                                                             │
│ Explain whether they would provide good diversification.                                                        │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
                                                                                                                   
  # Monthly returns in decimal format                                                                              
  returns_A = np.array([0.02, -0.01, 0.03, 0.01, -0.02, 0.04])                                                     
  returns_B = np.array([0.01, 0.00, 0.02, 0.02, -0.01, 0.03])                                                      
                                                                                                                   
  # Calculating the correlation coefficient                                                                        
  correlation = np.corrcoef(returns_A, returns_B)[0, 1]                                                            
  print("Correlation Coefficient:", correlation)                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Correlation Coefficient: 0.9286541181161709

Out: None

[Step 1: Duration 4.20 seconds| Input tokens: 2,041 | Output tokens: 218]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "correlation_coefficient": correlation,                                                                      
      "diversification": "poor diversification due to high positive correlation."                                  
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'correlation_coefficient': np.float64(0.9286541181161709), 'diversification': 'poor diversification 
due to high positive correlation.'}

[Step 2: Duration 2.65 seconds| Input tokens: 4,492 | Output tokens: 345]

---

## Combining Tools AND Code

Here's the really powerful thing: you can combine pre-built tools AND code generation.

In [8]:
@tool
def get_historical_returns(ticker: str) -> list:
    """Get historical monthly returns for a stock (last 12 months).

    Args:
        ticker: The stock symbol (AAPL, NVDA, or MSFT)

    Returns:
        List of monthly returns as decimals
    """
    # Simulated historical returns
    returns = {
        "AAPL": [0.02, 0.05, -0.03, 0.04, 0.01, 0.03, -0.02, 0.06, 0.02, -0.01, 0.04, 0.03],
        "NVDA": [0.08, 0.12, -0.05, 0.15, -0.03, 0.10, 0.07, -0.02, 0.09, 0.04, 0.11, 0.06],
        "MSFT": [0.03, 0.02, 0.01, 0.04, -0.01, 0.02, 0.03, 0.01, 0.02, 0.03, 0.01, 0.02]
    }
    return returns.get(ticker.upper(), [])

print("Tool created: get_historical_returns")

Tool created: get_historical_returns


In [9]:
# Agent with BOTH a tool AND code execution
agent_combined = CodeAgent(
    tools=[get_historical_returns],  # Has a tool for fetching data
    model=model,
    verbosity_level=LogLevel.INFO,
    additional_authorized_imports=["numpy", "pandas"]
)

print("Agent ready with tool + code generation!")

Agent ready with tool + code generation!


In [10]:
# Now the agent can fetch data with the tool AND compute custom metrics with code
result = agent_combined.run("""
Compare the Sharpe ratios of AAPL, NVDA, and MSFT.
Use a risk-free rate of 0.3% per month.
Rank them from best to worst risk-adjusted return.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare the Sharpe ratios of AAPL, NVDA, and MSFT.                                                              │
│ Use a risk-free rate of 0.3% per month.                                                                         │
│ Rank them from best to worst risk-adjusted return.                                                              │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
                                                                                                                   
  # Function to calculate the Sharpe Ratio                                                                         
  def calculate_sharpe_ratio(returns, risk_free_rate):                                                             
      average_return = np.mean(returns)                                                                            
      stddev_return = np.std(returns)                                                                              
      sharpe_ratio = (average_return - risk_free_rate) / stddev_return                                             
      return sharpe_ratio                                                                                          
                                                                                                                   
  # Retrieve historical returns for AAPL, NVDA, and MSFT                                                           
  aapl_returns = get_historical_returns("AAPL")                                                                    
  nvda_returns = get_historical_returns("NVDA")                                                                    
  msft_returns = get_historical_returns("MSFT")                                                                    
                                                                                                                   
  # Convert risk-free rate percentage to decimal                                                                   
  risk_free_rate = 0.003                                                                                           
                                                                                                                   
  # Calculate Sharpe ratios for each stock                                                                         
  sharpe_ratios = {                                                                                                
      "AAPL": calculate_sharpe_ratio(aapl_returns, risk_free_rate),                                                
      "NVDA": calculate_sharpe_ratio(nvda_returns, risk_free_rate),                                                
      "MSFT": calculate_sharpe_ratio(msft_returns, risk_free_rate),                                                
  }                                                                                                                
                                                                                                                   
  print("Sharpe Ratios:", sharpe_ratios)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Sharpe Ratios: {'AAPL': np.float64(0.6350242624919842), 'NVDA': np.float64(0.9391919122116633), 'MSFT': 
np.float64(1.2876232245039458)}

Out: None

[Step 1: Duration 6.53 seconds| Input tokens: 2,067 | Output tokens: 365]

...

[Step 2: Duration 3.05 seconds| Input tokens: 4,856 | Output tokens: 528]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({"MSFT": 1.288, "NVDA": 0.939, "AAPL": 0.635})                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'MSFT': 1.288, 'NVDA': 0.939, 'AAPL': 0.635}

[Step 3: Duration 2.90 seconds| Input tokens: 8,011 | Output tokens: 638]

**Best of both worlds:**
- The agent used the **tool** to get data
- It wrote **code** to calculate Sharpe ratios
- It **compared** and ranked them

---

## The Power and the Risk

CodeAct is incredibly powerful. But let's be honest about something:

**The agent is writing and executing code.** That means it could potentially do things you don't expect.

In a production environment, you'd want safeguards:
- Sandboxed execution environments
- Code review before execution
- Limited access to the file system
- Monitoring and logging

We cover all of this in the full course. For now, know that CodeAct is safe in these demo notebooks because we're using simulated data and limited imports.

---

## CodeAct Principles

### Principle 1: Be Specific About What You Want
The more context you give — formulas, constraints, output format — the better the generated code will be.

### Principle 2: Ask for Explanations
Add "show your work" or "explain the calculation" to your prompts. This makes the agent's reasoning visible.

### Principle 3: Combine Tools and Code
Use **tools for data fetching** (reliable, predictable) and **code for analysis** (flexible, custom). That's the pattern that scales.

---

## Your Turn: Exercise

Ask the agent to perform a financial calculation that would require custom code.

**Ideas:**
- "Calculate the Sortino ratio (like Sharpe but only penalizes downside volatility)"
- "Calculate Value at Risk (VaR) at the 95% confidence level"
- "Calculate the Calmar ratio (annualized return / max drawdown)"
- "Calculate the correlation between two stocks' returns"

Include the data in your prompt, or use the `get_historical_returns` tool.

In [ ]:
# EXERCISE: Ask the agent to perform a financial calculation

result = agent_combined.run("""
    YOUR PROMPT HERE
""")

---

## Bonus: Sortino Ratio Solution

In [ ]:
# Example: Sortino ratio calculation
result = agent_combined.run("""
Calculate the Sortino ratio for NVDA.

The Sortino ratio is like the Sharpe ratio, but it only penalizes
downside volatility (negative returns), not all volatility.

Formula: (Mean Return - Risk-Free Rate) / Downside Deviation

Where downside deviation = sqrt(mean of squared negative excess returns)

Use a risk-free rate of 0.3% per month.
Show your calculation steps.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the Sortino ratio for NVDA.                                                                           │
│                                                                                                                 │
│ The Sortino ratio is like the Sharpe ratio, but it only penalizes                                               │
│ downside volatility (negative returns), not all volatility.                                                     │
│                                                                                                                 │
│ Formula: (Mean Return - Risk-Free Rate) / Downside Deviation                                                    │
│                                                                                                                 │
│ Where downside deviation = sqrt(mean of squared negative excess returns)                                        │
│                                                                                                                 │
│ Use a risk-free rate of 0.3% per month.                                                                         │
│ Show your calculation steps.                                                                                    │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  historical_returns = get_historical_returns(ticker="NVDA")                                                       
  print("Historical Returns for NVDA:", historical_returns)                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Historical Returns for NVDA: [0.08, 0.12, -0.05, 0.15, -0.03, 0.1, 0.07, -0.02, 0.09, 0.04, 0.11, 0.06]

Out: None

[Step 1: Duration 2.93 seconds| Input tokens: 2,110 | Output tokens: 159]

...

[Step 3: Duration 3.28 seconds| Input tokens: 7,599 | Output tokens: 664]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(sortino_ratio)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 0.6937819061732103

[Step 4: Duration 1.25 seconds| Input tokens: 10,943 | Output tokens: 731]

---

## Recap

**What you learned:**

1. **CodeAct** lets agents write and execute Python code on the fly
2. It's powerful for **custom calculations** that you can't pre-build tools for
3. The agent combines **knowledge** (knowing formulas) with **execution** (running code)
4. **Combine tools and code:** tools for data, code for analysis

**Next up:** In Module 4, we'll see how to combine all three patterns — tool calling, ReAct, and CodeAct — into **orchestrated agents** that can handle complex, multi-stage tasks.